In [1]:
%pylab inline
import pymc3 as pm
import theano.tensor as tt

Populating the interactive namespace from numpy and matplotlib


In [2]:
# simulation of data. Two state model for simplicity.
N_seqs = 1000
N_labels = 3
N_states = 2

# transition probability
P = np.array([[0.8, 0.2], [0.4, 0.6]])
# emission probabilityu
Pe = np.array([
    [0.8, 0.1, 0.1],
    [0.3, 0.4, 0.3]
])
N_labels = Pe.shape[1]

AA = np.eye(N_states) - P + np.ones(shape=(N_states, N_states))
PA = np.linalg.solve(AA.T, np.ones(shape=(N_states)))

state_seq = [np.random.choice(N_states, p=PA)]
for i in range(1, N_seqs):
    state_seq += [np.random.choice(N_states, p=P[state_seq[-1]])]

label_seq = [np.random.choice(N_labels, p=Pe[state_seq[i]])
             for i in range(N_seqs)]

print(state_seq[:20])
print(label_seq[:20])

[0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 1, 0, 0, 0, 0, 0, 2, 2, 2, 0, 2, 1, 0, 0, 0, 1, 1, 1, 1]


In [12]:
with pm.Model() as model:
    P = pm.Dirichlet('P_transition', a=np.ones(
        (N_states, N_states)), shape=(N_states, N_states))

    AA = tt.eye(N_states) - P + tt.ones(shape=(N_states, N_states))
    PA = pm.Deterministic('P_start_transition', tt.slinalg.solve(
        AA.T, tt.ones(shape=(N_states))))

    states = tt.log(PA[state_seq[0]])
    for i in range(1, len(state_seq)):
        states += tt.log(P[state_seq[i-1]][state_seq[i]])
    pm.Potential('logp', states)
    trace = pm.sample(1000)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [P_transition]
Sampling 2 chains: 100%|██████████| 3000/3000 [00:03<00:00, 945.88draws/s] 


In [13]:
trace['P_transition'].mean(axis=0)

array([[0.77233287, 0.22766713],
       [0.40276056, 0.59723944]])

In [14]:
with pm.Model() as model:
    P = pm.Dirichlet('P_transition', 
                     a=np.ones((N_states, N_states)), 
                     shape=(N_states, N_states), 
                     testval=np.array([[0.8, 0.2], [0.4, 0.6]]))

    AA = tt.eye(N_states) - P + tt.ones(shape=(N_states, N_states))
    PA = pm.Deterministic('P_start_transition', tt.slinalg.solve(
        AA.T, tt.ones(shape=(N_states))))


In [15]:
P[state_seq[:-1]][np.arange(N_seqs-1), state_seq[1:]].tag.test_value.shape

(999,)

In [16]:
with model:
    pm.Potential('logp', 
                 tt.log(PA[state_seq[0]]) + 
                 tt.log(P[state_seq[:-1]][np.arange(N_seqs-1), state_seq[1:]]).sum())
    trace = pm.sample(1000)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [P_transition]
Sampling 2 chains: 100%|██████████| 3000/3000 [00:04<00:00, 708.55draws/s]


In [17]:
trace['P_transition'].mean(axis=0)

array([[0.77196573, 0.22803427],
       [0.40160257, 0.59839743]])

https://github.com/stan-dev/example-models/blob/master/misc/hmm/hmm-semisup.stan